CELL 1

In [3]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)


2.7.0


CELL 2

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52

Found existing installation: opencv-python-headless 4.5.5.62
Uninstalling opencv-python-headless-4.5.5.62:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.62.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-64ac49e1.so.58.91.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-4b79e479.so.58.45.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-805734e8.so.56.51.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-018b8c17.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dist

CELL 3 kết nối tới google drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 customTF2   Faces.jpg	'My Drive'   ODTF   test2.jpg   test.jpg


CELL 4 tải kho lưu trữ git mô hình tensorflow & Cài đặt API phát hiện đối tượng TensorFlow

In [5]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .


/content/models/research/models/research
Processing /content/models/research/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684828 sha256=96c8e2a3f2293fb575eb2dd0e6a18e174b6d8a4228fc2ef883ad9c27a3b32f72
  Stored in directory: /tmp/pip-ephem-wheel-cache-acpj4a31/wheels/c7/fa/f6/069042072998147abaf28ed1094914d80fae94930182fea2bc
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detecti

CELL 5 kiểm tra thử trình tạo model

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

2022-01-20 20:04:11.648320: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0120 20:04:12.082153 139980493272960 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.72s
I0120 20:04:12.373798 139980493272960 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.72s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.61s
I0120 20:04:12.979671 139980493272960 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_cr

CELL 6 giải nén folder chứa tệp dữ liệu

In [ ]:
%cd /mydrive/ODTF/data/


!unzip /mydrive/ODTF/images.zip -d .
!unzip /mydrive/ODTF/annotations.zip -d .

/content/gdrive/My Drive/ODTF/data
Archive:  /mydrive/ODTF/images.zip
  inflating: ./images/0.jpg          
  inflating: ./images/0-with-mask.jpg  
  inflating: ./images/100-with-mask.jpg  
  inflating: ./images/101-with-mask.jpg  
  inflating: ./images/103-with-mask.jpg  
  inflating: ./images/104-with-mask.jpg  
  inflating: ./images/105-with-mask.jpg  
  inflating: ./images/106-with-mask.jpg  
  inflating: ./images/107-with-mask.jpg  
  inflating: ./images/108-with-mask.jpg  
  inflating: ./images/109-with-mask.jpg  
  inflating: ./images/10-with-mask.jpg  
  inflating: ./images/110-with-mask.jpg  
  inflating: ./images/111-with-mask.jpg  
  inflating: ./images/112-with-mask.jpg  
  inflating: ./images/113-with-mask.jpg  
  inflating: ./images/114-with-mask.jpg  
  inflating: ./images/115-with-mask.jpg  
  inflating: ./images/116-with-mask.jpg  
  inflating: ./images/117-with-mask.jpg  
  inflating: ./images/118-with-mask.jpg  
  inflating: ./images/119-with-mask.jpg  
  inflating: 

CELL 7 
Tạo folder test_labels & train_labels

In [ ]:
# tạo 2 folder test_labels train_labels
!mkdir test_labels train_labels

# Di chuyển 274/1370 nhãn đầu tiên (20% số nhãn) sang dir thử nghiệm: `test_labels
!ls annotations/* | sort -R | head -274 | xargs -I{} mv {} test_labels/


# Di chuyển phần còn lại của các nhãn (1096 nhãn) sang dir đào tạo: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

CELL 8 Tạo tệp CSV và tệp "label_map.pbtxt"
Thư mục làm việc hiện tại là / mydrive / ODTF / data /

Chạy tập lệnh xml_to_csv bên dưới để tạo test_labels.csv và train_labels.csv

Điều này cũng tạo tệp label_map.pbtxt bằng cách sử dụng các lớp được đề cập trong các tệp xml.

In [ ]:
#adjusted from: https://github.com/hoanghung2504/ODTF/blob/main/xml_to_csv.py
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,   
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 



CELL 9 Tạo tệp train.record & test.record
Thư mục làm việc hiện tại là / mydrive / ODTF / data /

Chạy tập lệnh create_tfrecord.py để tạo các tệp train.record và test.record

In [ ]:
#For train.record
!python /mydrive/customTF2/generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record

#For test.record
!python /mydrive/customTF2/generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record


groups: 100% 1096/1096 [00:03<00:00, 352.77it/s]
Successfully created the TFRecords: /content/gdrive/My Drive/ODTF/data/train.record
groups: 100% 274/274 [00:00<00:00, 366.30it/s]
Successfully created the TFRecords: /content/gdrive/My Drive/ODTF/data/test.record


CELL 10 Tải xuống điểm kiểm tra mô hình được đào tạo trước
Thư mục làm việc hiện tại là / mydrive / ODTF / data /

Tải xuống ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz vào thư mục dữ liệu và giải nén.


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2022-01-20 17:31:58--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.13.128, 2a00:1450:400c:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.13.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  37.7MB/s    in 0.5s    

2022-01-20 17:31:59 (37.7 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-

Chỉnh sửa tệp cấu hình từ / models / research / object_detection / configs / tf2 trong colab và sao chép tệp cấu hình đã chỉnh sửa vào thư mục / mydrive / ODTF / data.

Bạn cần thực hiện các thay đổi sau:

thay đổi num_classes thành số lớp của bạn.
thay đổi đường dẫn test.record, đường dẫn train.record & đường dẫn nhãn bản đồ thành các đường dẫn mà bạn đã tạo các tệp này (các đường dẫn phải liên quan đến thư mục làm việc hiện tại của bạn trong khi đào tạo).
thay đổi fine_tune_checkpoint thành đường dẫn của thư mục chứa điểm kiểm tra đã tải xuống từ CELL 10.
thay đổi fine_tune_checkpoint_type với phân loại giá trị hoặc phát hiện tùy thuộc vào loại ..
thay đổi batch_size thành bội số 8 bất kỳ tùy thuộc vào khả năng của GPU của bạn. (ví dụ: - 24,128, ..., 512) .Mine được đặt thành 64.
thay đổi số bước thành số bước bạn muốn máy dò đào tạo. Sau đó sẽ chạy CELL 11

CELL 11

In [ ]:

#copy tệp cấu hình đã chỉnh sửa từ thư mục configs / tf2 vào data / folder trong ổ đĩa của bạn

!cp /content/models/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config /mydrive/ODTF/data

CELL 12: Training model

In [ ]:
%cd /content/models/research/object_detection
!python model_main_tf2.py --pipeline_config_path=/mydrive/ODTF/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config --model_dir=/mydrive/ODTF/training --alsologtostderr

/content/models/research/object_detection
2022-01-20 20:11:27.578090: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0120 20:11:27.579685 140250504361856 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0120 20:11:27.580880 140250504361856 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0120 20:11:27.587686 140250504361856 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0120 20:11:27.587892 140250504361856 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0120 20:11:27.617786 1402505043

CELL 13 Xuất model hoàn chỉnh

In [ ]:
# xuất model và lưu về drive
!python exporter_main_v2.py --trained_checkpoint_dir=/mydrive/ODTF/training --pipeline_config_path=/content/gdrive/MyDrive/ODTF/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config --output_directory /mydrive/ODTF/data/inference_graph